In [ ]:
import pandas as pd
import glob
import os
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import sklearn.ensemble
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, train_test_split, cross_val_predict, learning_curve
import sklearn.metrics

%matplotlib inline
%load_ext autoreload
%autoreload 2

# There's a lot of columns in the DF. 
# Therefore, we add this option so that we can see more columns
pd.options.display.max_columns = 100

In [ ]:
path = '../datas/nlp_results/'
voting_df = pd.read_csv(path+'voting_with_topics.csv')
print('Entries in the DataFrame',voting_df.shape)

#Dropping the useless column
voting_df = voting_df.drop('Unnamed: 0',1)

#Putting numerical values into the columns that should have numerical values
#print(voting_df.columns.values)

num_cols = ['Decision', ' armée', ' asile / immigration', ' assurances', ' budget', ' dunno', ' entreprise/ finance',
           ' environnement', ' famille / enfants', ' imposition', ' politique internationale', ' retraite  ']
voting_df[num_cols] = voting_df[num_cols].apply(pd.to_numeric)

#Inserting the full name at the second position
voting_df.insert(2,'Name', voting_df['FirstName'] + ' ' + voting_df['LastName'])

voting_df.head(3)

# Evolution of topics in the votes

We take a look at the data in order to observe which topics are most talked about in the votes, and see whether or not it changes over the years.

In [ ]:
voting_df_copy = voting_df
voting_df_copy.head(3)

In [ ]:
voting_df_copy['VoteEnd'] = [x[:4] for x in voting_df_copy['VoteEnd']]
voting_df_copy.head(3)
dates = voting_df_copy['VoteEnd'].drop_duplicates()
dates = np.sort(dates)
dates

In [ ]:
#voting_df_copy.index = voting_df_copy['VoteEnd']
voting_df_copy = voting_df_copy.set_index(['VoteEnd', 'Name'])

voting_df_copy.head(3)

In [ ]:
voting_df_copy.columns

In [ ]:
#voting_df_copy.loc['2009-09'][' armée']
#topicArmee = np.mean(voting_df_copy.loc['2009-09'][' armée'])

#print(topicArmee)
#indices = voting_df_copy.index[:].values
#indices

topics = ['armée', 'asile / immigration', 'assurances', 'budget', 'dunno', 'entreprise/ finance', 'environnement', 'famille / enfants', 'imposition', 'politique internationale', 'retraite  ']
topicEvolution = pd.DataFrame(index = dates, columns = topics)
#topicArmeeEvolution = []

for topic in topics:
    for date in dates:
        topicEvolution.loc[date][topic] = np.mean(voting_df_copy.loc[date][' '+topic])
        #topicArmeeEvolution.append(np.mean(voting_df_copy.loc[date][' '+topic]))
    #topicEvolution[topic].plot
    plt.plot(dates,topicEvolution[topic])

plt.legend(topics)
topicEvolution

In [ ]:
topicEvolution.to_json("topicEvolution.json")

# Ratio of acceptation for each topic

We now want to see if there is a significant correlation between a vote being accepted and the topic of the vote.

In [ ]:
voting_df_TopicAcceptation = voting_df
#voting_df_TopicAcceptation = voting_df_TopicAcceptation.set_index(['IdVote', 'Name'])
voting_df_TopicAcceptation.head(3)

In [ ]:
voting_df_TopicAcceptation['Topic'] = 'topic'

#voting_df_TopicAcceptation.loc[0]['Topic']
for i in range(voting_df_TopicAcceptation.shape[0]):
    voting_df_TopicAcceptation.loc[i]['Topic'] = 'topic modif'
    
voting_df_TopicAcceptation.head(3)